In [12]:
import math
def convert_to_float(s) -> float:
    if not isinstance(s, str):
        return math.nan  # Return NaN for non-string input
    
    parts = s.split('.')
    if len(parts) > 2:
        s = parts[0] + '.' + ''.join(parts[1:])  # Fix multiple dots
    
    try:
        return float(s)
    except ValueError:
        return math.nan  # Return NaN for invalid float conversion

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load dataset
df = pd.read_csv("data/Food_Time new.csv")  # Change to your actual dataset
import numpy as np

# Drop unnecessary columns
df_clean = df.drop(columns=["ID", "Delivery_person_ID"])

# Fix missing values
df_clean["Traffic_Level"].fillna(df_clean["Traffic_Level"].mode()[0], inplace=True)
df_clean["weather_description"].fillna(df_clean["weather_description"].mode()[0], inplace=True)
df_clean["temperature"].fillna(df_clean["temperature"].median(), inplace=True)
df_clean["humidity"].fillna(df_clean["humidity"].median(), inplace=True)
df_clean["precipitation"].fillna(df_clean["precipitation"].median(), inplace=True)
df_clean["Distance (km)"].fillna(df_clean["Distance (km)"].median(), inplace=True)
df_clean["TARGET"].fillna(df_clean["TARGET"].median(), inplace=True)

# Convert latitude/longitude columns to proper float values
lat_lon_cols = ["Restaurant_latitude", "Restaurant_longitude", "Delivery_location_latitude", "Delivery_location_longitude"]
for col in lat_lon_cols:
  df_clean[col] = df_clean[col].astype(str).str.replace(".", "", regex=False).astype(float) / 10**6  # Adjusting decimal placement

# Fix `TARGET` column formatting
df_clean["TARGET"] = df_clean["TARGET"].astype(str).replace(".", "", regex=False).astype(float) / 10**6

# Encode categorical variables
df_clean = pd.get_dummies(df_clean, drop_first=True)

# Remove duplicates
df_clean.drop_duplicates(inplace=True)

# Standardize numerical features
num_cols = df_clean.select_dtypes(include=["number"]).columns
df_clean[num_cols] = (df_clean[num_cols] - df_clean[num_cols].mean()) / df_clean[num_cols].std()

# Final check
df_clean.info(), df_clean.head()

# Fix incorrect number formatting in TARGET column
df_clean["TARGET"] = df_clean["TARGET"].astype(str).str.replace(r"[^\d.]", "", regex=True)

# Convert to float
df_clean["TARGET"] = pd.to_numeric(df_clean["TARGET"], errors="coerce")

# Retry filling missing values with median
df_clean["TARGET"].fillna(df_clean["TARGET"].median(), inplace=True)

# Verify the changes
df_clean["TARGET"].head()

from sklearn.preprocessing import StandardScaler

# Convert latitude/longitude columns to proper float values
for col in lat_lon_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

# Encode categorical variables using one-hot encoding
df_clean = pd.get_dummies(df_clean, drop_first=True)

# Remove duplicates
df_clean.drop_duplicates(inplace=True)

# Standardize numerical features
scaler = StandardScaler()
num_cols = df_clean.select_dtypes(include=["number"]).columns
df_clean[num_cols] = scaler.fit_transform(df_clean[num_cols])

# Final dataset check
df_clean.info(), df_clean.head()


TypeError: could not convert string to float: '3.816.666.667'

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8809 entries, 0 to 9999
Columns: 22471 entries, Delivery_person_Age to TARGET_99.85
dtypes: bool(22465), float64(6)
memory usage: 189.2 MB
